In [11]:
import pandas as pd
from typing import List, Dict

class ExcelToDetailedConverter:
    def __init__(self):
        self.input_data = []
        self.detailed_data = []

    def convert_excel_to_detailed(self, input_excel_path: str, output_excel_path: str):
        """
        요약 Excel 파일을 상세 Excel 파일로 변환
        
        Args:
            input_excel_path: log_analysis_result.xlsx 파일 경로
            output_excel_path: log_analysis_detailed.xlsx 출력 파일 경로
        """
        try:
            # 1. 입력 파일 읽기
            self._load_input_excel(input_excel_path)
            
            # 2. 상세 데이터로 변환
            self._convert_to_detailed()
            
            # 3. 출력 파일 저장
            self._save_detailed_excel(output_excel_path)
            
        except Exception as e:
            print(f"❌ 변환 중 오류 발생: {e}")

    def _load_input_excel(self, input_path: str):
        """입력 Excel 파일 로드"""
        try:
            df = pd.read_excel(input_path, engine='openpyxl')
            
            # 필수 컬럼 검증
            required_columns = ['page_id', 'click_type', 'act_type', 'key_list', 'value_list']
            missing_columns = [col for col in required_columns if col not in df.columns]
            
            if missing_columns:
                raise ValueError(f"필수 컬럼이 누락되었습니다: {missing_columns}")
            
            self.input_data = df.to_dict('records')
            print(f"✅ 입력 파일 로드 완료: {len(self.input_data)}개 로그")
            
        except FileNotFoundError:
            print(f"❌ 파일을 찾을 수 없습니다: {input_path}")
            raise
        except Exception as e:
            print(f"❌ 파일 로드 오류: {e}")
            raise

    def _convert_to_detailed(self):
        """요약 데이터를 상세 key-value 행으로 변환"""
        self.detailed_data = []
        
        for log_no, row in enumerate(self.input_data, 1):
            # 기본 정보 추출 (NaN을 공백으로 처리)
            page_id = self._clean_value(row.get('page_id', ''))
            click_type = self._clean_value(row.get('click_type', ''))
            act_type = self._clean_value(row.get('act_type', ''))
            
            # key_list와 value_list 파싱 (NaN을 공백으로 처리)
            key_list_str = self._clean_value(row.get('key_list', ''))
            value_list_str = self._clean_value(row.get('value_list', ''))
            
            # 쉼표로 분리하여 리스트로 변환
            keys = [k.strip() for k in key_list_str.split(',') if k.strip()] if key_list_str else []
            values = [self._clean_value(v.strip()) for v in value_list_str.split(',') if v.strip()] if value_list_str else []
            
            # key와 value 개수 맞추기
            min_length = min(len(keys), len(values))
            
            # 각 key-value 쌍을 개별 행으로 생성
            for i in range(min_length):
                key = keys[i]
                value = values[i]
                
                self.detailed_data.append({
                    'no': log_no,
                    'page_id': page_id,
                    'click_type': click_type,
                    'act_type': act_type,
                    'key': key,
                    'value': value
                })
            
            # key가 더 많은 경우 (value 없음)
            if len(keys) > len(values):
                for i in range(len(values), len(keys)):
                    self.detailed_data.append({
                        'no': log_no,
                        'page_id': page_id,
                        'click_type': click_type,
                        'act_type': act_type,
                        'key': keys[i],
                        'value': ''  # 공백으로 처리
                    })
        
        print(f"✅ 상세 데이터 변환 완료: {len(self.detailed_data)}개 key-value 쌍")

    def _clean_value(self, value):
        """NaN, None, 'nan' 값을 공백 문자열로 변환"""
        if pd.isna(value) or value is None:
            return ''
        
        # 문자열로 변환 후 'nan' 체크
        str_value = str(value)
        if str_value.lower() in ['nan', 'none', 'null']:
            return ''
        
        return str_value

    def _save_detailed_excel(self, output_path: str):
        """상세 데이터를 Excel 파일로 저장"""
        try:
            df = pd.DataFrame(self.detailed_data)
            
            # 컬럼 순서 정의
            column_order = ['no', 'page_id', 'click_type', 'act_type', 'key', 'value']
            df = df[column_order]
            
            # Excel 파일 저장
            df.to_excel(output_path, index=False, engine='openpyxl')
            
            print(f"✅ 상세 Excel 파일 저장: {output_path}")
            print(f"   총 {len(self.detailed_data)}개 key-value 쌍")
            print(f"   총 {len(self.input_data)}개 로그에서 변환됨")
            
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            raise

    def get_conversion_summary(self) -> Dict:
        """변환 결과 요약 정보 반환"""
        if not self.detailed_data:
            return {"error": "변환된 데이터가 없습니다."}
        
        # 통계 계산
        total_logs = len(self.input_data)
        total_key_values = len(self.detailed_data)
        avg_keys_per_log = total_key_values / total_logs if total_logs > 0 else 0
        
        # 가장 많은 키를 가진 로그 찾기
        key_counts = {}
        for item in self.detailed_data:
            log_no = item['no']
            key_counts[log_no] = key_counts.get(log_no, 0) + 1
        
        max_keys = max(key_counts.values()) if key_counts else 0
        min_keys = min(key_counts.values()) if key_counts else 0
        
        return {
            "total_logs": total_logs,
            "total_key_value_pairs": total_key_values,
            "average_keys_per_log": round(avg_keys_per_log, 2),
            "max_keys_in_single_log": max_keys,
            "min_keys_in_single_log": min_keys
        }

    def preview_data(self, num_rows: int = 10):
        """변환된 데이터 미리보기"""
        if not self.detailed_data:
            print("변환된 데이터가 없습니다.")
            return
        
        print(f"\n📋 상세 데이터 미리보기 (처음 {num_rows}개 행):")
        print("-" * 80)
        
        for i, item in enumerate(self.detailed_data[:num_rows]):
            print(f"{item['no']:3} | {item['page_id']:20} | {item['click_type']:10} | "
                  f"{item['act_type']:10} | {item['key']:15} | {item['value'][:30]}...")
        
        if len(self.detailed_data) > num_rows:
            print(f"... 외 {len(self.detailed_data) - num_rows}개 행")


def run_converter_example(input_excel_path,output_excel_path):
    """변환기 사용 예시"""
    converter = ExcelToDetailedConverter()
    
    # 변환 실행
    converter.convert_excel_to_detailed(
        input_excel_path=input_excel_path,
        output_excel_path=output_excel_path
    )
    
    # 결과 요약 출력
    summary = converter.get_conversion_summary()
    print(f"\n📊 변환 결과 요약:")
    for key, value in summary.items():
        print(f"   {key}: {value}")
    
    # 데이터 미리보기
    converter.preview_data(5)


if __name__ == "__main__":
    print("=== Excel 요약을 상세 형태로 변환 ===")
    run_converter_example(
        input_excel_path="./result/log_analysis_result_dropduplicate.xlsx",
        output_excel_path="./result/log_analysis_detailed.xlsx"
    )

=== Excel 요약을 상세 형태로 변환 ===
✅ 입력 파일 로드 완료: 2개 로그
✅ 상세 데이터 변환 완료: 26개 key-value 쌍
✅ 상세 Excel 파일 저장: ./result/log_analysis_detailed.xlsx
   총 26개 key-value 쌍
   총 2개 로그에서 변환됨

📊 변환 결과 요약:
   total_logs: 2
   total_key_value_pairs: 26
   average_keys_per_log: 13.0
   max_keys_in_single_log: 14
   min_keys_in_single_log: 12

📋 상세 데이터 미리보기 (처음 5개 행):
--------------------------------------------------------------------------------
  1 | ecommerce-dev/product/detail/21 |            | click      | channel         | Rround...
  1 | ecommerce-dev/product/detail/21 |            | click      | page_url        | https://ecommerce-dev.hectoinn...
  1 | ecommerce-dev/product/detail/21 |            | click      | page_id         | ecommerce-dev/product/detail/2...
  1 | ecommerce-dev/product/detail/21 |            | click      | act_type        | click...
  1 | ecommerce-dev/product/detail/21 |            | click      | prd_code        | 21...
... 외 21개 행
